# Carga Gold - Fato de Cotação

Este notebook realiza a carga da fato de cotação (fato_cotacao) a partir dos dados da tabela staging de cotação histórica.

## Imports

In [1]:
from spark_config import init_spark
from pyspark.sql import functions as F


## Start Spark Session

In [2]:
spark = init_spark("Carga fato cotação")


✅ Spark 3.5.7 iniciado com Hive local persistente!
📁 Warehouse: D:/Projetos/DataLake/spark-warehouse
📁 Metastore: D:/Projetos/DataLake/metastore_db



## Define Delta Table Paths

In [3]:
# Define caminhos locais onde serão armazenadas as tabelas Delta
base_silver_path = "D:/Projetos/Jornada_financas_pessoais/data/delta/silver"
base_gold_path = "D:/Projetos/Jornada_financas_pessoais/data/delta/gold"

# Define caminhos das tabelas Delta
delta_path_cotacao_historica = f"{base_silver_path}/stg_cotacao_historica"
delta_path_dim_ativo = f"{base_gold_path}/dim_ativo_financeiro"
delta_path_fato_cotacao = f"{base_gold_path}/fato_cotacao"

## Read Source Data

In [4]:
# Lê a tabela staging de cotação histórica
df_stg_cotacao_historica = spark.read.format("delta").load(delta_path_cotacao_historica)

print(f"[SUCESSO] Leitura da tabela staging em: {delta_path_cotacao_historica}")
print(f"Total de registros: {df_stg_cotacao_historica.count()}")

df_dim_ativo = spark.read.format("delta").load(delta_path_dim_ativo)

print(f"[SUCESSO] Leitura da tabela dimensão em: {delta_path_dim_ativo}")
print(f"Total de registros: {df_dim_ativo.count()}")

[SUCESSO] Leitura da tabela staging em: D:/Projetos/Jornada_financas_pessoais/data/delta/silver/stg_cotacao_historica
Total de registros: 14694933
[SUCESSO] Leitura da tabela dimensão em: D:/Projetos/Jornada_financas_pessoais/data/delta/gold/dim_ativo_financeiro
Total de registros: 3201


## Transform Data

In [5]:
# Filtra apenas registros com tp_mercado = '10'
df_stg_cotacao_historica = df_stg_cotacao_historica.filter(F.col("tp_mercado") == "010")

# Join LEFT (mantém todas as cotações mesmo sem correspondência na dimensão)
df_joined = (
    df_stg_cotacao_historica.alias("stg")
    .join(
        df_dim_ativo.alias("dim"),
        F.col("stg.cd_negociacao") == F.col("dim.cd_ativo"),
        "left"  # mantém as linhas da stg mesmo se não achar na dimensão
    )
)

# Tratamento da chave surrogate faltante (usa -1)
df_fato_cotacao = (
    df_joined.select(
        F.col("stg.dt_pregao"),
        F.when(F.col("sk_ativo").isNull(), F.lit("-1")).otherwise(F.col("sk_ativo")).alias("sk_ativo"),
        F.col("vl_abertura"),
        F.col("vl_minimo"),
        F.col("vl_maximo"),
        F.col("vl_medio"),
        F.col("vl_ultimo_negocio"),
        F.col("qt_negocios_efetuados").alias("qt_negocio"),
        F.col("qt_total_titulos").alias("qt_titulo"),
        F.col("vl_total_titulos").alias("vl_volume"),
        F.current_timestamp().alias("ts_insercao"),
        F.year(F.col("dt_pregao")).alias("nr_ano_pregao"),
        F.month(F.col("dt_pregao")).alias("nr_mes_pregao")
    )
)


## Write Data

In [7]:

df_fato_cotacao.write \
    .format("delta") \
    .mode("overwrite") \
    .option("overwriteSchema", "false") \
    .option("partitionOverwriteMode", "dynamic") \
    .partitionBy("nr_ano_pregao", "nr_mes_pregao") \
    .save(delta_path_fato_cotacao)

print("✅ Dynamic partition overwrite executado - apenas partições afetadas foram sobrescritas")

✅ Dynamic partition overwrite executado - apenas partições afetadas foram sobrescritas


## Stop Spark Session

In [ ]:
# Encerra a SparkSession
spark.stop()

NameError: name 'spark' is not defined